# Ejemplo con PyTorch

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [20]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # nueva capa

        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

        # Nuevas dimensiones después de 4 poolings: 32x32 → 16x16 → 8x8 → 4x4 → 2x2
        self.fc1 = nn.Linear(128 * 2 * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # (16, 16, 16)
        x = self.pool(self.relu(self.conv2(x)))  # (32, 8, 8)
        x = self.pool(self.relu(self.conv3(x)))  # (64, 4, 4)
        x = self.pool(self.relu(self.conv4(x)))  # (128, 2, 2)
        x = x.view(x.size(0), -1)                # Flatten (batch_size, 128*2*2)
        x = self.relu(self.fc1(x))               # ReLU extra
        x = self.relu(self.fc2(x))               # ReLU extra
        x = self.fc3(x)
        return x


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

Epoch 1, Loss: 1.5349905671252704
Epoch 2, Loss: 1.0783248399399217
Epoch 3, Loss: 0.881233480940701
Epoch 4, Loss: 0.7582948352386001
Epoch 5, Loss: 0.6745264438086417
Epoch 6, Loss: 0.596727002853014
Epoch 7, Loss: 0.5358238597129372
Epoch 8, Loss: 0.4783778405502219
Epoch 9, Loss: 0.4291055478436857
Epoch 10, Loss: 0.3894948085206331


In [23]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 71.82%
